In [1]:
import os
import json
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Path to your JSON files and images
json_dir = 'D:/ANAND Project/MATS Visco Clutch Rolling Operation/Final Model/Annotted/'
image_dir = 'D:/ANAND Project/MATS Visco Clutch Rolling Operation/Final Model/Dataset/'

# Label map for conversion
label_map = {'Okay': 1, 'Not Okay': 0}

# Lists to store image data and labels
image_data = []
image_labels = []

# Function to process JSON files and extract ROI
def process_json_files(json_dir, image_dir):
    for json_file in os.listdir(json_dir):
        if json_file.endswith('.json'):
            with open(os.path.join(json_dir, json_file)) as f:
                data = json.load(f)

            # Iterate through the entries in the JSON file
            for entry in data:
                img_path = entry['image']
                annotations = entry['annotations']
                
                # Process each annotation 
                for annotation in annotations:
                    label = annotation['label']  # 'Okay' or 'Not okay'
                    coordinates = annotation['coordinates']

                    # Extract the coordinates
                    x = int(coordinates['x'])
                    y = int(coordinates['y'])
                    width = int(coordinates['width'])
                    height = int(coordinates['height'])

                    # Load the corresponding image
                    img = cv2.imread(os.path.join(image_dir, img_path))

                    if img is not None:
                        # Crop the region of interest (ROI) based on the coordinates
                        roi = img[y:y + height, x:x + width]

                        # Resize the ROI for consistency (e.g., 224x224)
                        roi_resized = cv2.resize(roi, (224, 224))

                        # Normalize the ROI pixel values
                        roi_resized = roi_resized / 255.0

                        # Append the processed ROI and its label
                        image_data.append(roi_resized)
                        image_labels.append(label_map[label])

# Call the function to process all JSON files
process_json_files(json_dir, image_dir)

# Convert the lists to numpy arrays
image_data = np.array(image_data)
image_labels = np.array(image_labels)

# Convert labels to one-hot encoding
image_labels = to_categorical(image_labels, num_classes=2)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(image_data, image_labels, test_size=0.2, random_state=42)


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the CNN model
def create_model():
    model = Sequential()

    # First Convolutional Layer
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Second Convolutional Layer
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Third Convolutional Layer
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten the output
    model.add(Flatten())

    # Fully Connected Layer
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    # Output Layer (2 classes: 'Okay' and 'Not okay')
    model.add(Dense(2, activation='softmax'))

    # Compile model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Create model
model = create_model()

# Print model summary
model.summary()


C:\Users\Kunal\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │    22,151,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2)              │           514 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,245,186 (84.86 MB)

 Trainable params: 22,245,186 (84.86 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
# Train the model
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - accuracy: 0.5333 - loss: 0.8516 - val_accuracy: 0.9333 - val_loss: 0.3635
Epoch 2/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.9502 - loss: 0.2781 - val_accuracy: 1.0000 - val_loss: 0.0150
Epoch 3/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9852 - loss: 0.0534 - val_accuracy: 1.0000 - val_loss: 7.7962e-05
Epoch 4/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 1.0000 - loss: 0.0018 - val_accuracy: 1.0000 - val_loss: 1.7801e-06
Epoch 5/5
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 1.0000 - loss: 3.6426e-04 - val_accuracy: 1.0000 - val_loss: 1.3510e-07


In [4]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step - accuracy: 1.0000 - loss: 1.3510e-07
Test Accuracy: 100.00%


In [5]:
# Save the model
model.save('rollingcheck_model.h5')


In [4]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

# Load the trained model
model = load_model('rollingcheck_model.h5')

# Function to predict image orientation
def predict_orientation(image_path, coordinates):
    img = cv2.imread(image_path)
    
    x, y, width, height = int(coordinates['x']), int(coordinates['y']), int(coordinates['width']), int(coordinates['height'])
    roi = img[y:y + height, x:x + width]
    roi_resized = cv2.resize(roi, (224, 224))
    roi_resized = roi_resized / 255.0  # Normalize

    prediction = model.predict(np.expand_dims(roi_resized, axis=0))
    predicted_class = np.argmax(prediction, axis=1)

    return 'Rolling Present' if predicted_class == 1 else 'Rolling Not Present'

# Example 
coordinates_trial= {'x': 1425.0, 'y': 1725.0, 'width': 230.0, 'height': 1180.0}
image_path = 'D:/ANAND Project/MATS Visco Clutch Rolling Operation/Final Model/Trial UI/Rolling/IMG20250203101144.jpg'
result = predict_orientation(image_path, coordinates_trial) 
#Currently we are passing the coordinates, we wil set the coordinates after the fixed angle images.
print(f'Predicted Result: {result}')




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
Predicted Result: Rolling Present


# Okay 

In [57]:
# Example 
coordinates_trial= {'x': 1680.0, 'y': 2560.0, 'width': 230.0, 'height': 1180.0}
image_path = 'D:/ANAND Project/MATS Visco Clutch Rolling Operation/Final Model/Images/Not Okay/IMG20241015144711_BURST012.jpg'
result = predict_orientation(image_path, coordinates_trial) 
#Currently we are passing the coordinates, we wil set the coordinates after the fixed angle images.
print(f'Predicted Result: {result}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted Result: Rolling Present
